In [1]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from  pyspark.sql.functions import abs
from  pyspark.sql.functions import col
import time
from pyspark.sql.types import *

In [2]:
df = spark.read.load('hdfs://orion11:11101/3hr_sample/*',format='csv',sep='\t',inferSchema='true', header='true')

In [3]:
df.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface
0,1455440400000,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null
1,1455440400000,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null
2,1455440400000,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null
3,1455440400000,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null
4,1455440400000,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null


In [4]:
from math import log10

#  Note: the alphabet in geohash differs from the common base32 alphabet described in IETF's RFC 4648
# (http://tools.ietf.org/html/rfc4648)

__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
__decodemap = dict()
for i in range(len(__base32)):
    __decodemap[__base32[i]] = i


def decode_exactly(geohash):
    """
    Decode the geohash to its exact values, including the error
    margins of the result.  Returns four float values: latitude,
    longitude, the plus/minus error for latitude (as a positive
    number) and the plus/minus error for longitude (as a positive
    number).
    """
    lat_interval, lon_interval = (-90.0, 90.0), (-180.0, 180.0)
    lat_err, lon_err = 90.0, 180.0
    is_even = True
    for c in geohash:
        cd = __decodemap[c]
        for mask in [16, 8, 4, 2, 1]:
            if is_even: # adds longitude info
                lon_err /= 2
                if cd & mask:
                    lon_interval = ((lon_interval[0]+lon_interval[1])/2, lon_interval[1])
                else:
                    lon_interval = (lon_interval[0], (lon_interval[0]+lon_interval[1])/2)
            else:      # adds latitude info
                lat_err /= 2
                if cd & mask:
                    lat_interval = ((lat_interval[0]+lat_interval[1])/2, lat_interval[1])
                else:
                    lat_interval = (lat_interval[0], (lat_interval[0]+lat_interval[1])/2)
            is_even = not is_even
    lat = (lat_interval[0] + lat_interval[1]) / 2
    lon = (lon_interval[0] + lon_interval[1]) / 2
    return lat, lon, lat_err, lon_err


def decode(geohash):
    """
    Decode geohash, returning two float with latitude and longitude
    containing only relevant digits and with trailing zeroes removed.
    """
    lat, lon, lat_err, lon_err = decode_exactly(geohash)
    # Format to the number of decimals that are known
    lats = "%.*f" % (max(1, int(round(-log10(lat_err)))) - 1, lat)
    lons = "%.*f" % (max(1, int(round(-log10(lon_err)))) - 1, lon)
    if '.' in lats: lats = lats.rstrip('0')
    if '.' in lons: lons = lons.rstrip('0')
    return float(lats), float(lons)


def encode(latitude, longitude, precision=12):
    """
    Encode a position given in float arguments latitude, longitude to
    a geohash which will have the character count precision.
    """
    lat_interval = (-90.0, 90.0)
    lon_interval = (-180.0, 180.0)
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if longitude > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if latitude > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash += __base32[ch]
            bit = 0
            ch = 0
    return ''.join(geohash)

In [5]:
def convertToGeoHash(lat, lon):
    return encode(lat, lon,4)

udfConvertToGeoHash = F.udf(convertToGeoHash, StringType())

geohashDf = df.withColumn("5_hash", udfConvertToGeoHash("2_lat", "3_lon"))

In [6]:
geohashDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,1455440400000,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null,djph
1,1455440400000,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null,f4r8
2,1455440400000,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null,9qd2
3,1455440400000,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null,djvk
4,1455440400000,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null,c26u


In [7]:
def convertToMonth(value):
    return time.strftime('%m', time.localtime(int(value)/1000))

udfConvertToMonth = F.udf(convertToMonth, StringType())

monthDf = geohashDf.withColumn("1_time", udfConvertToMonth("1_time"))
monthDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,02,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null,djph
1,02,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null,f4r8
2,02,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null,9qd2
3,02,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null,djvk
4,02,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null,c26u


In [8]:
groupedDf = monthDf.groupBy("5_hash","1_time").agg(F.avg("temperature_surface").alias("temperature_surface"),F.avg("relative_humidity_zerodegc_isotherm").alias("relative_humidity_zerodegc_isotherm"),F.avg("pressure_surface").alias("pressure_surface"))

In [9]:
groupedDf.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,cbdz,03,269.587053,78.192982,98885.922061
1,95b6,03,295.399960,25.488189,101888.240012
2,9xkh,03,272.791013,73.562500,77616.651554
3,9mue,03,290.076991,28.016216,101159.419546
4,c830,03,270.862677,79.110759,81874.272811


In [10]:
comfortIdxDf = groupedDf.filter(groupedDf.temperature_surface > 293).filter(groupedDf.temperature_surface < 304)\
        .filter(groupedDf.relative_humidity_zerodegc_isotherm < 25).filter(groupedDf.relative_humidity_zerodegc_isotherm > 5)\
        .filter(groupedDf.pressure_surface > 101000).filter(groupedDf.pressure_surface < 102000)

In [11]:
comfortIdxDf.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,95n4,10,299.893973,24.636364,101238.664248
1,95v2,10,297.972161,15.514377,101483.772562
2,952z,10,298.949692,18.886792,101472.489645
3,958e,10,298.661261,17.403292,101526.046621
4,9kdx,10,295.254002,15.789773,101495.515256


In [12]:
#temperature_surface (298 - a)
#relative_humidity_zerodegc_isotherm (15 - b)
#pressure_surface(101500 - c)
# from  pyspark.sql.functions import *
from  pyspark.sql.functions import abs

def getComfortIndex(temperature_surface, relative_humidity_zerodegc_isotherm, pressure_surface):
    if temperature_surface < 298:
        x = 298 - temperature_surface
    else:
        x = temperature_surface -298
        
    if relative_humidity_zerodegc_isotherm < 15:
        y = 15 - relative_humidity_zerodegc_isotherm
    else:
        y = relative_humidity_zerodegc_isotherm -15
        
    if pressure_surface < 101500:
        z = 101500 - pressure_surface
    else:
        z = pressure_surface -101500
    
    return (x + y + z)/3

udfConvertToComfortIndex = F.udf(getComfortIndex, StringType())

# label is the comfort index
comfortIndexPreMLDf = comfortIdxDf.withColumn("label", udfConvertToComfortIndex("temperature_surface", "relative_humidity_zerodegc_isotherm","pressure_surface"))

In [13]:
comfortIndexPreMLDf.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface,label
0,9h78,01,294.452656,23.495050,101995.828162,169.29018509351368
1,d7sg,01,300.251162,18.260870,101680.127528,61.879853467119574
2,d7gv,01,300.146123,19.618421,101758.587022,88.45052201754335
3,9ejs,01,301.055458,23.722022,101357.754852,51.340875896514056
4,d6xv,01,300.708492,10.327913,101493.586211,4.598122393859918


In [14]:
comfortIndexPreMLDf.columns[2:-1]

['temperature_surface',
 'relative_humidity_zerodegc_isotherm',
 'pressure_surface']

In [15]:
# ML Analysis begins here

In [16]:
for col_name in comfortIndexPreMLDf.columns[2:]:
    comfortIndexPreMLDf3 = comfortIndexPreMLDf.withColumn(col_name,col(col_name).cast('float'))

In [17]:
comfortIndexPreMLDf3.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface,label
0,9kts,12,293.521624,20.829932,101738.396786,82.901695
1,d5de,12,301.269737,24.305921,101600.328526,37.634727
2,95ys,12,295.523142,20.266667,101838.426671,115.390068
3,9k94,12,293.576559,19.700000,101964.923307,158.015579
4,9krm,12,295.818871,18.223837,101607.346228,37.583733


In [18]:
comfortIndexPreMLDf3.dtypes

[('5_hash', 'string'),
 ('1_time', 'string'),
 ('temperature_surface', 'double'),
 ('relative_humidity_zerodegc_isotherm', 'double'),
 ('pressure_surface', 'double'),
 ('label', 'float')]

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [20]:
comfortIndexPreMLDf3.columns

['5_hash',
 '1_time',
 'temperature_surface',
 'relative_humidity_zerodegc_isotherm',
 'pressure_surface',
 'label']

In [21]:
comfortIndexPreMLDf3.columns[2:-1]

['temperature_surface',
 'relative_humidity_zerodegc_isotherm',
 'pressure_surface']

In [22]:
vectorAssembler = VectorAssembler(inputCols=comfortIndexPreMLDf3.columns[2:-1], outputCol="features")
df_tr = vectorAssembler.transform(comfortIndexPreMLDf3)

In [23]:
train, test = df_tr.randomSplit([0.8, 0.2])

In [24]:
lr = LinearRegression(featuresCol="features",labelCol="label")

In [25]:
#from pyspark.ml.linalg import Vectors, VectorUDT

In [26]:
model = lr.fit(train)

In [27]:
evaluation_summary = model.evaluate(test)

In [28]:
evaluation_summary

In [29]:
df_tr = df_tr.select(['features','label'])
df_tr.show(3)

+--------------------+---------+
|            features|    label|
+--------------------+---------+
|[294.452655940594...|169.29019|
|[300.251162408026...|61.879852|
|[300.146123070175...| 88.45052|
+--------------------+---------+
only showing top 3 rows



In [30]:
predictions = model.transform(test)

In [31]:
predictions.select(predictions.columns[2:]).show()

+-------------------+-----------------------------------+------------------+---------+--------------------+------------------+
|temperature_surface|relative_humidity_zerodegc_isotherm|  pressure_surface|    label|            features|        prediction|
+-------------------+-----------------------------------+------------------+---------+--------------------+------------------+
| 296.22063469453377|                 16.177419354838708|101923.21336977495|142.05672|[296.220634694533...|111.92702392093997|
| 296.71955547101464|                 12.384057971014492|101853.76842753626| 119.2216|[296.719555471014...| 94.04187596494194|
| 298.46329262135924|                  24.41747572815534| 101505.2668867314| 5.049218|[298.463292621359...|58.459319863150085|
| 297.64160194346294|                   21.5354609929078|101599.75398939928|35.549282|[297.641601943462...| 69.82517927896333|
|  297.1748498310811|                 22.408783783783782|101705.24762500002| 71.16052|[297.174849831081...| 87.

In [32]:
predictionsDf = predictions.select(predictions.columns[2:])

In [33]:
predictionsDf.dtypes

[('temperature_surface', 'double'),
 ('relative_humidity_zerodegc_isotherm', 'double'),
 ('pressure_surface', 'double'),
 ('label', 'float'),
 ('features', 'vector'),
 ('prediction', 'double')]

In [34]:
predictions.limit(10).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface,label,features,prediction
0,8gzy,12,296.220635,16.177419,101923.213370,142.056717,"[296.2206346945337, 16.177419354838708, 101923...",111.927024
1,8uz3,09,296.719555,12.384058,101853.768428,119.221603,"[296.71955547101464, 12.384057971014492, 10185...",94.041876
2,94vd,01,298.463293,24.417476,101505.266887,5.049218,"[298.46329262135924, 24.41747572815534, 101505...",58.459320
3,9502,05,297.641602,21.535461,101599.753989,35.549282,"[297.64160194346294, 21.5354609929078, 101599....",69.825179
4,957j,12,297.174850,22.408784,101705.247625,71.160522,"[297.1748498310811, 22.408783783783782, 101705...",87.936976
5,95mr,06,296.890109,21.059441,101500.000136,2.389822,"[296.89010888501747, 21.05944055944056, 101500...",56.988019
6,95q7,10,299.549408,18.752212,101303.936881,67.121582,"[299.54940818584066, 18.75221238938053, 101303...",16.619060
7,95v2,10,297.972161,15.514377,101483.772562,5.589885,"[297.9721608945686, 15.514376996805112, 101483...",42.040685
8,95vk,04,294.879772,14.097015,101850.227153,118.083458,"[294.8797724999999, 14.097014925373134, 101850...",102.103678
9,95vw,01,294.921826,22.915858,101920.599589,143.864548,"[294.92182611650486, 22.915857605177994, 10192...",126.822073


In [50]:
predictions.filter(col("prediction") > 0).orderBy("prediction").limit(50).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface,label,features,prediction
0,d59p,03,302.253561,21.873720,101213.002048,99.375076,"[302.2535606122449, 21.87372013651877, 101213....",0.316200
1,d595,02,299.489860,14.224138,101243.159142,86.368858,"[299.48986025862064, 14.224137931034482, 10124...",0.493028
2,d7c2,12,299.114679,19.547170,101173.996596,110.555084,"[299.1146791698113, 19.547169811320753, 101173...",0.566266
3,9s0k,10,300.366827,24.282486,101146.269633,121.793228,"[300.3668269209039, 24.282485875706215, 101146...",0.576066
4,9gr7,02,300.468413,17.326923,101232.086224,90.903038,"[300.4684128205127, 17.326923076923077, 101232...",1.030939
5,9krx,10,297.766653,19.037618,101156.214085,116.018959,"[297.7666534482759, 19.037617554858933, 101156...",1.341473
6,9mu9,10,294.313441,18.353933,101092.581843,138.152878,"[294.31344112359557, 18.353932584269664, 10109...",1.758056
7,9mx5,11,293.287542,21.047059,101041.966565,156.264313,"[293.2875415294117, 21.04705882352941, 101041....",2.140748
8,9s2s,11,296.013142,24.889610,101056.293636,151.860947,"[296.01314220779216, 24.88961038961039, 101056...",2.145625
9,9gnm,03,300.484448,24.010811,101163.775432,115.906609,"[300.4844477837838, 24.01081081081081, 101163....",2.280823
